In [8]:
import torch
import torch.nn.functional as F
print(torch.__version__)

# The PyG built-in GCNConv
from torch_geometric.nn import GCNConv

import torch_geometric.transforms as T
from ogb.nodeproppred import PygNodePropPredDataset, Evaluator

1.9.1


In [9]:
#dataset
dataset_name = 'ogbn-arxiv'
dataset = PygNodePropPredDataset(name=dataset_name,
                                 transform=T.ToSparseTensor())

data = dataset[0]
print(data)

data.adj_t=data.adj_t.to_symmetric()
split_idx=dataset.get_idx_split()
device = 'cuda' if torch.cuda.is_available() else 'cpu'
train_idx=split_idx['train'].to(device)


Data(x=[169343, 128], node_year=[169343, 1], y=[169343, 1], adj_t=[169343, 169343, nnz=1166243])


Model
![picture](cs224w-colab2-3.png)

In [10]:
#models <-Construct the network as showing in the figure
class GCN(torch.nn.Module):
  def __init__(self, input_dim, hidden_dim,output_dim,num_layers,dropout):
    super(GCN, self).__init__()
    self.convs=None
    self.bns=None
    self.dropout=dropout
    self.num_layers=num_layers
    self.convs=torch.nn.ModuleList()
    self.convs.append(GCNConv(input_dim, hidden_dim))
    for i in range(num_layers-2):
      self.convs.append(GCNConv(hidden_dim,hidden_dim))
    self.convs.append(GCNConv(hidden_dim,output_dim))
    self.bns=torch.nn.ModuleList([torch.nn.BatchNorm1d(hidden_dim) for i in range(num_layers-1)])
    self.softmax=torch.nn.LogSoftmax()
  
  def reset_parameters(self):
    for conv in self.convs:
      conv.reset_parameters()
    for bn in self.bns:
      bn.reset_parameters()
  def forward(self,x, adj_t):
    for i in range(self.num_layers-1):
      x=self.convs[i](x, adj_t)
      x=self.bns[i](x)
      x=F.relu(x)
      x=F.dropout(x,self.dropout,self.training)
    x=self.convs[-1](x,adj_t)
    x=self.softmax(x)
    return x


In [11]:
print(data.x.shape)
#取多行
print(data.x[train_idx].shape)
#取一行 降维
print(data.x[0].shape)
#取一列 降维
print(data.x[:,0].shape)
print(data.x[train_idx,0].shape)


torch.Size([169343, 128])
torch.Size([90941, 128])
torch.Size([128])
torch.Size([169343])
torch.Size([90941])


Train

In [12]:
def train(model, data, train_idx, optimizer, loss_fn):
    # TODO: Implement this function that trains the model by 
    # using the given optimizer and loss_fn.
    model.train()
    loss = 0

    ############# Your code here ############
    ## Note:
    ## 1. Zero grad the optimizer
    ## 2. Feed the data into the model
    ## 3. Slicing the model output and label by train_idx
    ## 4. Feed the sliced output and label to loss_fn
    ## (~4 lines of code)
    optimizer.zero_grad()
    #x=data.x[train_idx]
    y=data.y[train_idx,0]
    #why  model(data.x[train_idx]) is wrong ?
    l=loss_fn(model(data.x,data.adj_t)[train_idx],y)
    l.backward()
    optimizer.step()
    return l.item()

In [18]:
# Test function here
@torch.no_grad()
def test(model, data, split_idx, evaluator):
    # TODO: Implement this function that tests the model by 
    # using the given split_idx and evaluator.
    model.eval()
    #Sets the module in evaluation mode.

    # The output of model on all data
    out = None

    ############# Your code here ############
    ## (~1 line of code)
    ## Note:
    ## 1. No index slicing here
    out=model(data.x,data.adj_t)
    #########################################
    y_pred=torch.argmax(out,dim=-1,keepdim=True)
    train_acc = evaluator.eval({
        'y_true': data.y[split_idx['train']],
        'y_pred': y_pred[split_idx['train']],
    })['acc']
    valid_acc = evaluator.eval({
        'y_true': data.y[split_idx['valid']],
        'y_pred': y_pred[split_idx['valid']],
    })['acc']
    test_acc = evaluator.eval({
        'y_true': data.y[split_idx['test']],
        'y_pred': y_pred[split_idx['test']],
    })['acc']
    return train_acc, valid_acc, test_acc
  

In [20]:
#hyperparameters
args = {
    'device': device,
    'num_layers': 3,
    'hidden_dim': 256,
    'dropout': 0.5,
    'lr': 0.01,
    'epochs': 100,
}
#instanize
model = GCN(data.num_features, args['hidden_dim'],
            dataset.num_classes, args['num_layers'],
            args['dropout']).to(device)
evaluator = Evaluator(name='ogbn-arxiv')
args

{'device': 'cpu',
 'num_layers': 3,
 'hidden_dim': 256,
 'dropout': 0.5,
 'lr': 0.01,
 'epochs': 100}

In [15]:
#debug module is
#model(data.x,data.adj_t)

In [16]:
optimizer=torch.optim.Adam(model.parameters(),args['lr'])
loss_fn=F.nll_loss
#train(model,data,train_idx, optimizer, loss_fn)

<ipython-input-10-80373950429f>:29: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  x=self.softmax(x)


4.260563850402832

Training

In [21]:
#train
import copy
# reset the parameters to initial random value
model.reset_parameters()
optimizer=torch.optim.Adam(model.parameters(),args['lr'])
loss_fn=F.nll_loss

best_acc=0
best_model=None
for epoch in range(args['epochs']):
  loss=train(model,data,train_idx, optimizer, loss_fn)
  train_acc, valid_acc, test_acc=test(model,data,split_idx,evaluator)
  if valid_acc>best_acc:
    best_acc=valid_acc
    best_model=copy.deepcopy(model)
  print(f'epoch:{epoch:02d},',f'train_acc:{train_acc:.2f},',f'valid_acc:{valid_acc:.2f}',f'test_acc:{test_acc:.2f}')

<ipython-input-10-80373950429f>:29: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  x=self.softmax(x)


epoch:00, train_acc:0.14, valid_acc:0.24 test_acc:0.22
epoch:01, train_acc:0.26, valid_acc:0.23 test_acc:0.28
epoch:02, train_acc:0.24, valid_acc:0.24 test_acc:0.24
epoch:03, train_acc:0.22, valid_acc:0.16 test_acc:0.15
epoch:04, train_acc:0.26, valid_acc:0.16 test_acc:0.15
epoch:05, train_acc:0.30, valid_acc:0.19 test_acc:0.18
epoch:06, train_acc:0.36, valid_acc:0.27 test_acc:0.26
epoch:07, train_acc:0.42, valid_acc:0.34 test_acc:0.34
epoch:08, train_acc:0.45, valid_acc:0.38 test_acc:0.37
epoch:09, train_acc:0.46, valid_acc:0.41 test_acc:0.41
epoch:10, train_acc:0.46, valid_acc:0.45 test_acc:0.47
epoch:11, train_acc:0.45, valid_acc:0.44 test_acc:0.46
epoch:12, train_acc:0.45, valid_acc:0.43 test_acc:0.44
epoch:13, train_acc:0.46, valid_acc:0.43 test_acc:0.43
epoch:14, train_acc:0.48, valid_acc:0.46 test_acc:0.45
epoch:15, train_acc:0.52, valid_acc:0.50 test_acc:0.49
epoch:16, train_acc:0.55, valid_acc:0.54 test_acc:0.54
epoch:17, train_acc:0.57, valid_acc:0.56 test_acc:0.58
epoch:18, 

graph level
dataprocess